In [2]:
#import packages
from libchebipy import ChebiEntity
import libchebipy
import pubchempy as pcp
import io
import requests
import json
import cobra
import functions
from functions import *

In [3]:
model=cobra.io.load_matlab_model('Data/iFpraus_v_1_0.mat')
info(model)

Metabolites :  833
Reactions :  1030
Genes :  602
Compartments :  {'c': '', 'e': ''}
Objective function : 
 Maximize
1.0*Biomass_FP - 1.0*Biomass_FP_reverse_ee33b 



---
## mMCB

(There are also supplements mentioned in the paper but this is the base medium)
* [Bacteriological Peptome (6.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN): 
    * Vague, decided to go with polypeptides: <b>polypep</b>
    * Nitrogen: <b>n2</b>
* [Soy Peptome (5.0) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0044&cat=&c=UK&lang=EN) --> another vague one:
    * Stachyose: <b>stys</b>
    * Raffinose: <b>raffin</b>
    * Sucrose: <b>sucr</b>
    * Nitrogen: <b>n2</b>
* [Yeast Extract (3.0) (VWR International, Darmstadt, Germany)](https://us.vwr.com/store/product/7437401/vwr-life-science-yeast-extract-bacteriological-grade):
    * Vitamin B (same logic as RCM):
        * 1. Thiamin -> <b>thm</b>
        * 2. Riboflavin -> <b>ribflv</b>
        * 3. Niacin -> <b>trp__L</b> (made of tryptophan)
        * 5. Pantothenic Acid -> <b>pnto__R</b>
        * 6. Pyridoxine -> <b>pydxn</b>
        * 7. Biotin -> <b>btn</b>
        * 9. Folic Acid -> ... can't find anything
        * 12. Cobalamin -> <b>b12</b> (or cbl1)
* [Tryptone (2.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0042&c=UK&lang=EN): 
    * Tryptophan: <b>trp__L</b>
* NaCL (1.5) (VWR International, Darmstadt, Germany):
    * NaCl: <b>na1, cl</b>
* K<sub>2</sub>HPO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* KH<sub>2</sub>PO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* Na<sub>2</sub>SO<sub>4</sub> (2.0) (VWR):
    * Sodium: <b>na1</b>
    * Sulfate: <b>so4</b>
* MgSO<sub>4</sub>*7H<sub>2</sub>O (1.0) (Merck):
    * Magnesium: <b>mg2</b>
    * Sulfate: <b>so4</b>
* CaCl<sub>2</sub>*2H<sub>2</sub>O (0.1) (Merck):
    * Calcium Chloride: <b>ca2, cl</b>
* NH<sub>4</sub>Cl (1.0) (Merck):
    * Ammonium Chloride: <b>nh4, cl</b>
* Cysteine-HCL (0.4) (Merck):
    * Cysteine: <b>cys__L</b>
    * HCL: <b>h2, cl</b>
* NaHCO<sub>3</sub> (0.2) (VWR):
    * Sodium: <b>na1</b>
    * Bicarbonate: <b>hco3<b>
* MnSO<sub>4</sub>*H<sub>2</sub>O (0.05) (VWR):
    * Manganese: <b>mn2</b>
    * Sulfate: <b>so4</b>
* FeSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (Merck):
    * Iron: <b>fe</b>
    * Sulfate: <b>so4</b>
* ZnSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (VWR):
    * Zinc: <b>zn2</b>
    * Sulfate: <b>so4</b>
* Hemin (0.005) (Sigma-Aldrich, Steinheim, Germany):
    * ... contains iron
* Menadione (0.005) (S-A):
    * <b>mndn</b>
* Resazurin (0.001) (S-A):
    * Fluoro identifier



| Component                            | Concentration (g/L) |
|--------------------------------------|---------------------|
| Bacteriological Peptome              | 6.5                 |
| Soy Peptome                          | 5.0                 |
| Yeast Extract                        | 3.0                 |
| Tryptone                             | 2.5                 |
| NaCL                                 | 1.5                 |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 |
| NH<sub>4</sub>Cl                     | 1.0                 |
| Cysteine-HCL                         | 0.4                 |
| NaHCO<sub>3</sub>                    | 0.2                 |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               |
| Hemin                                | 0.005               |
| Menadione                            | 0.005               |
| Resazurin                            | 0.001               |

In [4]:
# Get medium from model reactions
root_model = model.copy()
current_medium = [metab.lstrip('EX_') for metab in list(root_model.medium.keys())]

# CHeck which metabolites control growth by removing one at a time and seeing the results
def growth_stoppers(model, metabolites):
    results = {}
    for metabolite in metabolites:
        metabolites.remove(metabolite.lstrip('EX_'))
        test_model = medium(model, metabolites)
        results[metabolite.lstrip('EX_')] = test_model.slim_optimize()
        metabolites = [metab.lstrip("EX_") for metab in metabolites]
        metabolites.append(metabolite.lstrip('EX_'))

    import pandas as pd
    results_df = pd.DataFrame(results.items(), columns=["metab","growth"]).sort_values('growth', ascending=False)

    return results_df

growth_check_df = growth_stoppers(root_model, current_medium)

# Don't take these out of the medium if you can help it:
mandatory = growth_check_df[growth_check_df['growth'] <= 11.03796]
mandatory

,metab,growth
42,cps_fp(e),1.077058e+01
64,glyc3p(e),5.518982e+00
119,pi(e),5.518982e+00
110,nac(e),7.879200e-13
141,trp_L(e),6.209833e-15
87,k(e),3.821436e-15
41,cobalt2(e),3.598136e-15
43,cu2(e),3.598136e-15
50,fol(e),1.340104e-15
131,ser_L(e),-2.866077e-15


In [30]:
root_model.reactions.get_by_id('EX_ala_D(e)')

Reaction identifier,EX_ala_D(e)
Name,D-Alanine exchange
Memory address,0x07f865fd61280
Stoichiometry,ala-D[e] <=> D-Alanine <=>
GPR,
Lower bound,[-0.11744593]
Upper bound,1000


In [5]:
### Remove metabolites from the medium that can't be justified
current_medium.remove('EX_4abz(e)')
current_medium.remove('EX_5oxpro(e)')
current_medium.remove('EX_ac(e)')
current_medium.remove('EX_acasp(e)')
current_medium.remove('EX_acgal(e)')
current_medium.remove('EX_acgalglcur(e)')
current_medium.remove('EX_acgam(e)')
current_medium.remove('EX_acglu(e)')
current_medium.remove('EX_acnam(e)')
current_medium.remove('EX_adocbl(e)')
# current_medium.remove('EX_ala_D(e)')
# current_medium.remove('EX_alaala(e)')
# current_medium.remove('EX_alaasp(e)')
# current_medium.remove('EX_alagln(e)')
# current_medium.remove('EX_alaglu(e)')
# current_medium.remove('EX_alagly(e)')
# current_medium.remove('EX_alahis(e)')
# current_medium.remove('EX_alaleu(e)')
# current_medium.remove('EX_alathr(e)')
current_medium.remove('EX_aso3(e)')
current_medium.remove('EX_aso4(e)')
current_medium.remove('EX_but(e)')
current_medium.remove('EX_butso3(e)')
current_medium.remove('EX_cd2(e)')
# current_medium.remove('EX_cellb(e)')
# current_medium.remove('EX_cgly(e)')
current_medium.remove('EX_cit(e)')
# current_medium.remove('EX_cobalt2(e)') ###
# current_medium.remove('EX_cps_fp(e)') ###
current_medium.remove('EX_dhor_S(e)')
current_medium.remove('EX_ethso3(e)')
current_medium.remove('EX_fe3dcit(e)')
current_medium.remove('EX_for(e)')
current_medium.remove('EX_fru(e)')
current_medium.remove('EX_fum(e)')
# current_medium.remove('EX_gal(e)')
# current_medium.remove('EX_galur(e)')
# current_medium.remove('EX_glcur(e)')
# current_medium.remove('EX_glyasn(e)')
# current_medium.remove('EX_glyasp(e)')
# current_medium.remove('EX_glyc3p(e)') ###
# current_medium.remove('EX_glyc_R(e)')
# current_medium.remove('EX_glygln(e)')
# current_medium.remove('EX_glyglu(e)')
# current_medium.remove('EX_glygly(e)')
# current_medium.remove('EX_glyleu(e)')
# current_medium.remove('EX_glymet(e)')
# current_medium.remove('EX_glyphe(e)')
# current_medium.remove('EX_glypro(e)')
# current_medium.remove('EX_glytyr(e)')
current_medium.remove('EX_gthox(e)')
current_medium.remove('EX_gthrd(e)')
# current_medium.remove('EX_h2s(e)')
current_medium.remove('EX_hg2(e)')
current_medium.remove('EX_his_L(e)')
current_medium.remove('EX_hxan(e)')
# current_medium.remove('EX_ile_L(e)')
current_medium.remove('EX_ins(e)')
current_medium.remove('EX_inulin(e)')
current_medium.remove('EX_isetac(e)')
current_medium.remove('EX_kesto(e)')
current_medium.remove('EX_kestopt(e)')
current_medium.remove('EX_kestottr(e)')
# current_medium.remove('EX_lac_D(e)')
# current_medium.remove('EX_lcts(e)')
# current_medium.remove('EX_leu_L(e)')
# current_medium.remove('EX_leugly(e)')
# current_medium.remove('EX_leuleu(e)')
# current_medium.remove('EX_lys_L(e)')
current_medium.remove('EX_mal_L(e)')
# current_medium.remove('EX_malt(e)')
# current_medium.remove('EX_meoh(e)')
# current_medium.remove('EX_met_D(e)')
# current_medium.remove('EX_met_L(e)')
# current_medium.remove('EX_metala(e)')
# current_medium.remove('EX_metsox_R_L(e)')
# current_medium.remove('EX_metsox_S_L(e)')
current_medium.remove('EX_mobd(e)')
current_medium.remove('EX_mso3(e)')
# current_medium.remove('EX_nac(e)') ###
current_medium.remove('EX_ncam(e)')
current_medium.remove('EX_o2(e)')
# current_medium.remove('EX_orn(e)')
# current_medium.remove('EX_orot(e)')
# current_medium.remove('EX_pb(e)')
# current_medium.remove('EX_pect(e)') ###
# current_medium.remove('EX_phe_L(e)')
current_medium.remove('EX_plac(e)')
# current_medium.remove('EX_pro_L(e)')
# current_medium.remove('EX_progly(e)')
current_medium.remove('EX_ptrc(e)')
# current_medium.remove('EX_pydam(e)')
# current_medium.remove('EX_pydx(e)') ###
current_medium.remove('EX_seln(e)')
# current_medium.remove('EX_so4(e)') ###
current_medium.remove('EX_spmd(e)')
# current_medium.remove('EX_strch1(e)')
current_medium.remove('EX_succ(e)')
current_medium.remove('EX_sulfac(e)')
# current_medium.remove('EX_taur(e)')
current_medium.remove('EX_thymd(e)')
# current_medium.remove('EX_tyr_L(e)')
current_medium.remove('EX_urate(e)')
current_medium.remove('EX_urea(e)')
# current_medium.remove('EX_val_L(e)')
current_medium.remove('EX_xan(e)')
current_medium.remove('EX_xyl_D(e)')

In [6]:
# Get molar weights from Chebi
x=[root_model.reactions.get_by_id(compound_id).name for compound_id in current_medium]
x=[compound.rstrip("exchange") for compound in x]
";".join(x)

url = "http://api.xialab.ca/mapcompounds"

payload = "{\n\t\"queryList\": \"%s;\",\n\t\"inputType\": \"name\"\n}" %";".join(x)
headers = {
    'Content-Type': "application/json",
    'cache-control': "no-cache",
    }

response = requests.request("POST", url, data=payload, headers=headers)

dict=json.loads(response.text)
mapped_compounds=pd.DataFrame(dict)

mol_weight=[]
for i in mapped_compounds["chebi_id"]:
    try:
        x= ChebiEntity(str(i))
        mol_weight.append(x.get_mass())
    except:
        mol_weight.append("NA")
        continue

mapped_compounds["mol_weight"]=mol_weight
mapped_compounds["BiGG_ID"]=[root_model.reactions.get_by_id(compound_id).id.lstrip("EX_") for compound_id in current_medium]
weights=mapped_compounds[["query", "BiGG_ID", "hit", "mol_weight"]]
# weight: g/mol
# flux: mmol/g/hr

In [7]:
# Fill in any additional missing weights using Chebi synonyms & CIDs
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
for metab in weights.loc[weights['mol_weight'] == 'NA', 'query']:
    try:
        cid = pcp.get_synonyms(metab, 'name')[0]['CID']
        weights.loc[weights['query'] == metab, 'mol_weight'] = pcp.Compound.from_cid(cid).molecular_weight
    except:
        pass

# Still remaining
weights[weights['mol_weight'] == 'NA']

,query,BiGG_ID,hit,mol_weight
26,"Capsular polysaccharide (F. prausnitzii, putat...",cps_fp(e),,NA
30,Fe3,fe3(e),,NA
46,exchange reaction for glycylglyci,glygly(e),,NA
49,exchange reaction for glycylphenylalai,glyphe(e),,NA
61,exchange reaction for leucylglyci,leugly(e),,NA
79,exchange reaction for pectins,pect(e),,NA
92,"starch, structure 1 (1,6-{7[1,4-Glc], 4[1,4-Gl...",strch1(e),,NA


In [8]:
### MANUALLY fill these in
weights.loc[weights['BiGG_ID'] == 'fe3(e)', 'mol_weight'] = 55.85
weights.loc[weights['BiGG_ID'] == 'glygly(e)', 'mol_weight'] = 75.07*2
weights.loc[weights['BiGG_ID'] == 'glyphe(e)', 'mol_weight'] = 75.07+165.19
weights.loc[weights['BiGG_ID'] == 'leugly(e)', 'mol_weight'] = 75.07+131.17
weights.loc[weights['BiGG_ID'] == 'pect(e)', 'mol_weight'] = 194.14
weights.loc[weights['BiGG_ID'] == 'strch1(e)', 'mol_weight'] = 359.33
weights.loc[weights['BiGG_ID'] == 'cps_fp(e)', 'mol_weight'] = 1000
# Cobalt was Co2 somehow???
weights.loc[weights['BiGG_ID'] == 'cobalt2(e)', 'mol_weight'] = 129.839

# Still remaining
weights[weights['mol_weight'] == 'NA']

,query,BiGG_ID,hit,mol_weight


In [9]:
# http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN&minfo=Y
traces_list = [{'BiGG_ID':'ca2(e)', "conc_ppm_Bact_Peptone_L37": 635, "conc_ppm_Proteose_Peptone_L85": 200, "conc_ppm_soya_peptone_L44": 225, "conc_ppm_yeast_extract": 155},
{'BiGG_ID':'mg2(e)', "conc_ppm_Bact_Peptone_L37": 265, "conc_ppm_Proteose_Peptone_L85": 340, "conc_ppm_soya_peptone_L44": 1530, "conc_ppm_yeast_extract": 205},
{'BiGG_ID':'fe(e)', "conc_ppm_Bact_Peptone_L37": 22, "conc_ppm_Proteose_Peptone_L85": 42, "conc_ppm_soya_peptone_L44": 90, "conc_ppm_yeast_extract": 52},
{'BiGG_ID':'cu2(e)', "conc_ppm_Bact_Peptone_L37": 1, "conc_ppm_Proteose_Peptone_L85": 10, "conc_ppm_soya_peptone_L44": 2, "conc_ppm_yeast_extract": 2},
{'BiGG_ID':'pb(e)', "conc_ppm_Bact_Peptone_L37": 0.4, "conc_ppm_Proteose_Peptone_L85": 0.1, "conc_ppm_soya_peptone_L44": 0.3, "conc_ppm_yeast_extract": 0.7},
{'BiGG_ID':'mn2(e)', "conc_ppm_Bact_Peptone_L37": 3.4, "conc_ppm_Proteose_Peptone_L85": 0.5, "conc_ppm_soya_peptone_L44": 1.0, "conc_ppm_yeast_extract": 1.3},
{'BiGG_ID':'zn2(e)', "conc_ppm_Bact_Peptone_L37": 9.2, "conc_ppm_Proteose_Peptone_L85": 16, "conc_ppm_soya_peptone_L44": 12, "conc_ppm_yeast_extract": 94},
{'BiGG_ID':'cobalt2(e)', "conc_ppm_Bact_Peptone_L37": 0.1, "conc_ppm_Proteose_Peptone_L85": 0.2, "conc_ppm_soya_peptone_L44": 0.2, "conc_ppm_yeast_extract": 3.1},
{'BiGG_ID':'cl(e)', "conc_ppm_Bact_Peptone_L37": 10000, "conc_ppm_Proteose_Peptone_L85": 80000, "conc_ppm_soya_peptone_L44": 4000, "conc_ppm_yeast_extract": 3000},
{'BiGG_ID':'k(e)', "conc_ppm_Bact_Peptone_L37": 36000, "conc_ppm_Proteose_Peptone_L85": 14000, "conc_ppm_soya_peptone_L44": 33000, "conc_ppm_yeast_extract": 73000},
{'BiGG_ID':'na1(e)', "conc_ppm_Bact_Peptone_L37": 10000, "conc_ppm_Proteose_Peptone_L85": 80000, "conc_ppm_soya_peptone_L44": 4000, "conc_ppm_yeast_extract": 3000}]


aa_list = [{'BiGG_ID':'ala', "conc_ppm_Bact_Peptone_L37": 39200 , "conc_ppm_Proteose_Peptone_L85": 38100, "conc_ppm_soya_peptone_L44": 28700, "conc_ppm_yeast_extract": 9100},
{'BiGG_ID':'arg', "conc_ppm_Bact_Peptone_L37": 49900, "conc_ppm_Proteose_Peptone_L85": 58000, "conc_ppm_soya_peptone_L44": 46400, "conc_ppm_yeast_extract": 33100},
{'BiGG_ID':'asp', "conc_ppm_Bact_Peptone_L37": 60600, "conc_ppm_Proteose_Peptone_L85": 58500, "conc_ppm_soya_peptone_L44": 70600, "conc_ppm_yeast_extract": 70700}, 
{'BiGG_ID':'cys', "conc_ppm_Bact_Peptone_L37": 16600, "conc_ppm_Proteose_Peptone_L85": 1500, "conc_ppm_soya_peptone_L44": 5300, "conc_ppm_yeast_extract": 7600},
{'BiGG_ID':'glu', "conc_ppm_Bact_Peptone_L37": 99300, "conc_ppm_Proteose_Peptone_L85": 137800, "conc_ppm_soya_peptone_L44": 147100, "conc_ppm_yeast_extract": 134900},
{'BiGG_ID':'gly', "conc_ppm_Bact_Peptone_L37": 77100 , "conc_ppm_Proteose_Peptone_L85": 44500, "conc_ppm_soya_peptone_L44": 28300, "conc_ppm_yeast_extract": 59500},
{'BiGG_ID':'ile', "conc_ppm_Bact_Peptone_L37": 38100, "conc_ppm_Proteose_Peptone_L85": 45800, "conc_ppm_soya_peptone_L44": 25100, "conc_ppm_yeast_extract": 48100},
{'BiGG_ID':'leu', "conc_ppm_Bact_Peptone_L37": 37900, "conc_ppm_Proteose_Peptone_L85": 60100, "conc_ppm_soya_peptone_L44": 43100, "conc_ppm_yeast_extract": 60400},
{'BiGG_ID':'lys', "conc_ppm_Bact_Peptone_L37": 43800, "conc_ppm_Proteose_Peptone_L85": 46100, "conc_ppm_soya_peptone_L44": 37700, "conc_ppm_yeast_extract": 54000},
{'BiGG_ID':'met', "conc_ppm_Bact_Peptone_L37": 15800, "conc_ppm_Proteose_Peptone_L85": 10800, "conc_ppm_soya_peptone_L44": 6200, "conc_ppm_yeast_extract": 8000},
{'BiGG_ID':'phe', "conc_ppm_Bact_Peptone_L37": 26000, "conc_ppm_Proteose_Peptone_L85": 46600, "conc_ppm_soya_peptone_L44": 3800, "conc_ppm_yeast_extract": 37800},
{'BiGG_ID':'pro', "conc_ppm_Bact_Peptone_L37": 58300, "conc_ppm_Proteose_Peptone_L85": 59900, "conc_ppm_soya_peptone_L44": 34000, "conc_ppm_yeast_extract": 8800},
{'BiGG_ID':'ser', "conc_ppm_Bact_Peptone_L37": 28100, "conc_ppm_Proteose_Peptone_L85": 21800, "conc_ppm_soya_peptone_L44": 6700, "conc_ppm_yeast_extract": 34200},
{'BiGG_ID':'thr', "conc_ppm_Bact_Peptone_L37": 12500, "conc_ppm_Proteose_Peptone_L85": 27500, "conc_ppm_soya_peptone_L44": 16800, "conc_ppm_yeast_extract": 27300},
{'BiGG_ID':'trp', "conc_ppm_Bact_Peptone_L37": 6600, "conc_ppm_Proteose_Peptone_L85": 7500, "conc_ppm_soya_peptone_L44": 6400, "conc_ppm_yeast_extract": 8500},
{'BiGG_ID':'tyr', "conc_ppm_Bact_Peptone_L37": 3900, "conc_ppm_Proteose_Peptone_L85": 17700, "conc_ppm_soya_peptone_L44": 20900, "conc_ppm_yeast_extract": 49500},
{'BiGG_ID':'val', "conc_ppm_Bact_Peptone_L37": 33300, "conc_ppm_Proteose_Peptone_L85": 41100, "conc_ppm_soya_peptone_L44": 36500, "conc_ppm_yeast_extract": 10000}]

medium_trace = pd.DataFrame(traces_list)
medium_aa = pd.DataFrame(aa_list)

In [10]:
medium_aa.head()

,BiGG_ID,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract
0,ala,39200,38100,28700,9100
1,arg,49900,58000,46400,33100
2,asp,60600,58500,70600,70700
3,cys,16600,1500,5300,7600
4,glu,99300,137800,147100,134900


In [11]:

weights_and_traces = weights.merge(medium_trace, how="left", on="BiGG_ID")
weights_and_traces.head()

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract
0,L-cysteate,Lcyst(e),Cysteic acid,169.156,NaN,NaN,NaN,NaN
1,L-Cystine,Lcystin(e),L-Cystine,240.303,NaN,NaN,NaN,NaN
2,Adenine,ade(e),Adenine,135.127,NaN,NaN,NaN,NaN
3,D-Alanine,ala_D(e),D-Alanine,89.0932,NaN,NaN,NaN,NaN
4,L-Alanine,ala_L(e),L-Alanine,89.0932,NaN,NaN,NaN,NaN


In [31]:

def split_concentration_proportions(df_to_update, df_aa_medium):
    # Usage: splits concentrations in each medium for each amino acid and assigns an equal proportion to related amino acids
    # ie. cys__L = 16600; cyscys = NA --> cys__L = 8300, cyscys = 8300
    df_to_update = df_to_update.fillna(0)

    for aa in df_aa_medium["BiGG_ID"].values:
        aa_list = df_to_update.loc[df_to_update["BiGG_ID"].str.contains(aa), "BiGG_ID"]
        aa_count = len(aa_list)

        for media in df_aa_medium.columns[1:]:
            proportion = (df_aa_medium.loc[df_aa_medium["BiGG_ID"] == aa, media]/aa_count).values[0]
            for related_aa in aa_list:
                df_to_update.loc[df_to_update["BiGG_ID"]==related_aa, media] += proportion
    return df_to_update

weights_full = split_concentration_proportions(weights_and_traces, medium_aa)
weights_full.head()

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract
0,L-cysteate,Lcyst(e),Cysteic acid,169.15600,5533.333333,500.000000,1766.666667,2533.333333
1,L-Cystine,Lcystin(e),L-Cystine,240.30256,5533.333333,500.000000,1766.666667,2533.333333
2,Adenine,ade(e),Adenine,135.12690,0.000000,0.000000,0.000000,0.000000
3,D-Alanine,ala_D(e),D-Alanine,89.09322,3563.636364,3463.636364,2609.090909,827.272727
4,L-Alanine,ala_L(e),L-Alanine,89.09322,3563.636364,3463.636364,2609.090909,827.272727


In [18]:
# Assign proportions based on mmcb medium concentrations:
# | Component                            | Concentration (g/L) | BiGG Metabolites                                |
# |--------------------------------------|---------------------|-------------------------------------------------|
# | Bacteriological Peptome              | 6.5                 | polypep, n2                                     |
# | Soy Peptome                          | 5.0                 | stys, raffin, sucr, n2                          |
# | Yeast Extract                        | 3.0                 | thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
# | Tryptone                             | 2.5                 | trp\__L                                         |
# | NaCL                                 | 1.5                 | na1, cl                                         |
# | K<sub>2</sub>PO<sub>4</sub>          | 1.0                 | k, p1, h2                                       |
# | KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 | k, p1, h2                                       |
# | Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 | na1, so4                                        |
# | Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 | mg2, so4                                        |
# | CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 | ca2, cl                                         |
# | NH<sub>4</sub>Cl                     | 1.0                 | nh4, cl                                         |
# | Cysteine-HCL                         | 0.4                 | cys__L, h2, cl                                  |
# | NaHCO<sub>3</sub>                    | 0.2                 | na1, hco3                                       |
# | MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                | mn2, so4                                        |
# | FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               | fe, so4                                         |
# | ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               | zn2, so4                                        |
# | Hemin                                | 0.005               |                                                 |
# | Menadione                            | 0.005               | mndn                                            |
# | Resazurin                            | 0.001               |                                                 |

# weights_full
peptone_columns = {"conc_ppm_Bact_Peptone_L37": 6.233, "conc_ppm_Proteose_Peptone_L85": 0, "conc_ppm_soya_peptone_L44": 5.233, "conc_ppm_yeast_extract": 3.233}
trp_update = {'trp': 2.5}
supplements = {'na1(e)': 1.26, 'cl(e)': 0.566, 'pi(e)': 0.66, 'k(e)': 0.66, 'h(e)': 0.66, 'so4(e)': 0.66, 'mg2(e)': 0.66, 'ca2(e)': 0.033, 'nh4(e)': 1.0, 'fe2(e)': 0.000625, 'fe3(e)': 0.000625}

# Scale by peptone columns
# Update trp metabolites
# Add supplements

44

In [19]:
# Still lots of 0's...
len(weights_full[weights_full["conc_ppm_Bact_Peptone_L37"] == 0])
# Try the avg concentrations of the rest?
for media in weights_full.columns[4:]:
    fill_value = weights_full[media].mean()
    weights_full[media] = weights_full[media].replace(0, fill_value)


In [20]:
weights_full['g/L'] = weights_full.iloc[:,4:8].sum(axis=1)/1000
weights_full['mmol/L'] = weights_full['g/L']/weights_full['mol_weight']
weights_full.head()

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract,g/L,mmol/L
0,L-cysteate,Lcyst(e),Cysteic acid,169.15600,5533.333333,500.000000,1766.666667,2533.333333,10.333333,0.061088
1,L-Cystine,Lcystin(e),L-Cystine,240.30256,5533.333333,500.000000,1766.666667,2533.333333,10.333333,0.043001
2,Adenine,ade(e),Adenine,135.12690,7039.015000,8978.503000,6063.575000,7408.658000,29.489751,0.218237
3,D-Alanine,ala_D(e),D-Alanine,89.09322,3563.636364,3463.636364,2609.090909,827.272727,10.463636,0.117446
4,L-Alanine,ala_L(e),L-Alanine,89.09322,3563.636364,3463.636364,2609.090909,827.272727,10.463636,0.117446


,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract,g/L,mmol/L
0,L-cysteate,Lcyst(e),Cysteic acid,169.15600,5533.333333,500.000000,1766.666667,2533.333333,10.333333,0.061088
1,L-Cystine,Lcystin(e),L-Cystine,240.30256,5533.333333,500.000000,1766.666667,2533.333333,10.333333,0.043001
2,Adenine,ade(e),Adenine,135.12690,7039.015000,8978.503000,6063.575000,7408.658000,29.489751,0.218237
3,D-Alanine,ala_D(e),D-Alanine,89.09322,3563.636364,3463.636364,2609.090909,827.272727,10.463636,0.117446
4,L-Alanine,ala_L(e),L-Alanine,89.09322,3563.636364,3463.636364,2609.090909,827.272727,10.463636,0.117446
5,D-Alanyl-D-alanine,alaala(e),D-Alanyl-D-alanine,160.17116,3563.636364,3463.636364,2609.090909,827.272727,10.463636,0.065328
6,L-alanyl-L-aspartate,alaasp(e),Alanyl-Aspartic acid,203.17000,23763.636364,22963.636364,26142.424242,24393.939394,97.263636,0.478730
7,L-alanyl-L-glutamine,alagln(e),Alanyl-Glutamine,217.22000,3563.636364,3463.636364,2609.090909,827.272727,10.463636,0.048171
8,L-alanyl-L-glutamate,alaglu(e),Alanyl-Glutamic acid,218.21000,36663.636364,49396.969697,51642.424242,45793.939394,183.496970,0.840919
9,L-alanylglycine,alagly(e),Alanylglycine,146.14000,8098.930481,6081.283422,4273.796791,4327.272727,22.781283,0.155887


In [25]:
for id in ["EX_"+metab for metab in list(weights_full['BiGG_ID'])]:
    root_model.reactions.get_by_id(id).upper_bound = 1000
    root_model.reactions.get_by_id(id).lower_bound = weights_full.loc[weights_full["BiGG_ID"] == id.lstrip("EX_"), "mmol/L"].values*-1
root_model.objective = {root_model.reactions.get_by_id('Biomass_FP'): 1}
root_model.optimize()

,fluxes,reduced_costs
26DAPLLATi,0.002101,-3.155444e-30
3HAD100,0.005102,0.000000e+00
3HAD120,0.003041,0.000000e+00
3HAD121,0.002061,0.000000e+00
3HAD140,0.002901,1.577722e-30
...,...,...
XYLt2,0.000000,-5.430102e-16
YUMPS,0.000000,0.000000e+00
r0502,-0.001147,-6.110657e-16
r0839,0.000000,-3.417149e-15
